<a href="https://colab.research.google.com/github/conorlime/CS6271/blob/main/Copy%20of%20CS6271%20Kaggle%20Competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
<img src="http://hive.csis.ul.ie/UL_logo.jpeg" width="350"/>
</div> 

#**Artificial Intelligence - MSc**
##CS6271 - Evolutionary Algorithms and Humanoid Robotics 
### Kaggle Competition


Leader: Conor Ryan

Developer: Enrique Naredo

Tester: Allan De Lima

## Introduction

This notebook contains bare-bones functionality to generate either GP or GE models and submit the results to Kaggle. We will be using the **United States census dataset**.

The [United States census](https://en.wikipedia.org/wiki/United_States_census) is a census that is legally mandated by the US Constitution, and takes place every 10 years. 
* The first census after the American Revolution was taken in 1790, under Secretary of State Thomas Jefferson; there have been 23 federal censuses since that time.

In [ ]:
# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Dataset

This one of the availables USA census dataset and provides 14 input variables that are a mixture of categorical, ordinal, and numerical data types:

1. Age
2. Workclass
3. Final Weight
4. Education
5. Education Number of Years
6. Marital-status
7. Occupation
8. Relationship
9. Race
10. Sex
11. Capital-gain
12.Capital-loss
13. Hours-per-week
14. Native-country


This is a binary classification problem where the task is to predict whether a given adult makes more than $\$50,000$. 
* Class 0 $<= \$50,000$, 
* Class 1 $> \$50,000$.

### Load the dataset

In [ ]:
# Suppressing Warnings:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
## mount your Google drive
# 1) click on the link
# 2) sign in
# 3) copy the provided code
# 4) paste it in the text box bellow
# 5) click the folder icon at the right
# 6) verify your drive is mounted

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Clone the GRAPE repository at first because the dataset to be used is already there.

In [ ]:
import os
# Get the library from our BDS research Group
# copy the path from your drive
PATH = '/content/drive/MyDrive/grape/'

# check if 'grape' already exists
if os.path.exists(PATH):
    print('grape directory already exists')
else:
    %cd /content/drive/MyDrive/
    !git clone https://github.com/UL-BDS/grape.git 
    print('Cloning grape in your Drive')

# change directory to 'grape'
%cd /content/drive/MyDrive/grape/

grape directory already exists
/content/drive/MyDrive/grape


### Train set

In [ ]:
train_file = 'datasets/USA_census_train.csv'

In [ ]:
# load train set
df_train = pd.read_csv(PATH+train_file)
df_train.head()

,id,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week:,native-country,class_label
0,0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [ ]:
df_train.describe()

,id,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week:
count,1000.000000,1000.000000,1.000000e+03,1000.00000,1000.000000,1000.000000,1000.000000
mean,541.981000,37.940000,1.921667e+05,10.19800,567.717000,102.534000,40.706000
std,314.257019,12.896927,1.079408e+05,2.50181,2432.128661,431.964552,11.802918
min,0.000000,17.000000,2.117400e+04,1.00000,0.000000,0.000000,1.000000
25%,270.750000,28.000000,1.154045e+05,9.00000,0.000000,0.000000,40.000000
50%,543.500000,37.000000,1.808920e+05,10.00000,0.000000,0.000000,40.000000
75%,816.250000,46.000000,2.471522e+05,13.00000,0.000000,0.000000,45.000000
max,1084.000000,90.000000,1.033222e+06,16.00000,25236.000000,2415.000000,99.000000


In [ ]:
X_train = df_train.copy()
# warning: cannot drop it more than once
X_train.drop(['class_label'], axis=1, inplace=True)

In [ ]:
# class labels
l, _ = X_train.shape

y_train = np.zeros([l,], dtype=float)

for i in range(l):
  y_train[i] = 1 if df_train['class_label'].iloc[i] == ">50K" else 0

In [ ]:
#y_train.head()
print(y_train[0:5])

[0. 0. 0. 0. 0.]


### Test set

In [ ]:
test_file = 'datasets/USA_census_test.csv'

In [ ]:
# load test set
df_test = pd.read_csv(PATH+test_file)
df_test.head()

,id,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week:,native-country
0,1085,30,Private,241259,Assoc-voc,11,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States
1,1086,32,Private,118161,HS-grad,9,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
2,1087,29,Private,201954,Bachelors,13,Never-married,Exec-managerial,Not-in-family,White,Male,0,0,35,United-States
3,1088,42,Private,150533,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,7298,0,52,United-States
4,1089,38,Private,412296,HS-grad,9,Divorced,Craft-repair,Own-child,White,Male,0,0,28,United-States


In [ ]:
df_test.describe()

,id,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week:
count,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000
mean,1321.360465,40.020930,192969.458140,10.258140,1534.923256,94.181395,41.979070
std,136.103528,12.586189,105554.118494,2.362462,9868.029117,408.643454,11.289248
min,1085.000000,17.000000,22154.000000,2.000000,0.000000,0.000000,5.000000
25%,1204.250000,30.000000,120862.500000,9.000000,0.000000,0.000000,40.000000
50%,1321.500000,40.000000,182582.000000,10.000000,0.000000,0.000000,40.000000
75%,1439.750000,49.000000,241324.250000,13.000000,0.000000,0.000000,45.000000
max,1552.000000,88.000000,795830.000000,16.000000,99999.000000,2339.000000,99.000000


In [ ]:
X_test = df_test.copy()

We need to prepare both training and test datasets before working with a Machine Learning method. 

Consider the following tips:

1.   Normalise real data;
2.   Ignore columns that you think does not influence the class label (for example 'id');
3.   Use some encoding method with categorical data.

You are free to use other pre-processing ideas, and these don't only have to be ones we used in the module.

You are **strongly** advised to familiarise yourself with Segment 5.04 on Sulis before attempting this.




Number of categories on each categorical data:



1.   workclass: 7
2.   education: 16
3.   marital-status: 7
4. occupation: 14
5. relationship: 6
6. race: 5
7. native-country: 41



In [ ]:
X_train.dtypes

id                  int64
age                 int64
workclass          object
fnlwgt              int64
education          object
education-num       int64
marital-status     object
occupation         object
relationship       object
race               object
sex                object
capital-gain        int64
capital-loss        int64
hours-per-week:     int64
native-country     object
dtype: object

In [ ]:
X_train.head()

,id,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week:,native-country
0,0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


Normalise the data. Note that there is non-numeric data here, so that is why we will get NaN when we examine the results later on. For this example, we will ignore non-numeric data.

In [ ]:
# normalising numeric data    
features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week:', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

X_train.loc[:, features] = (X_train.loc[:, features] - X_train.loc[:, features].mean())/X_train.loc[:, features].std()
X_test.loc[:, features] = (X_test.loc[:, features] - X_test.loc[:, features].mean())/X_test.loc[:, features].std()

X_train.head()

,id,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week:,native-country
0,0,0.082190,NaN,-1.062162,NaN,1.119989,NaN,NaN,NaN,NaN,NaN,0.660443,-0.237367,-0.059816,NaN
1,1,0.935106,NaN,-1.008476,NaN,1.119989,NaN,NaN,NaN,NaN,NaN,-0.233424,-0.237367,-2.347386,NaN
2,2,0.004652,NaN,0.217520,NaN,-0.478853,NaN,NaN,NaN,NaN,NaN,-0.233424,-0.237367,-0.059816,NaN
3,3,1.167720,NaN,0.394237,NaN,-1.278274,NaN,NaN,NaN,NaN,NaN,-0.233424,-0.237367,-0.059816,NaN
4,4,-0.770726,NaN,1.354837,NaN,1.119989,NaN,NaN,NaN,NaN,NaN,-0.233424,-0.237367,-0.059816,NaN


Convert the datasets to NumPy to easily use them.

In [ ]:
# data features
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()


## GRAPE

<div>
<img src="http://hive.csis.ul.ie/GRAPE.png" width="150"/>
</div> 

GRammatical Algorithms in Python for Evolution (GRAPE)


Import adapted functions and classes from PonyGE2.

Import our algorithms, adapted from deap.algorithms.

Import DEAP (maybe it can be necessary to run the installation before that).



In [ ]:
!pip install deap==1.3 

from ponyge2_adapted_files import Grammar, ge
import algorithms

from os import path

from deap import creator, base, tools

     |████████████████████████████████| 152 kB 5.0 MB/s 


You can import functions to be used with your grammar from [functions.py](https://github.com/UL-BDS/grape/blob/main/functions.py) on GRAPE repository and / or you can define your own functions.

In [ ]:
from functions import add, sub, mul, pdiv#, psqrt, plog, neg, and_, or_, not_, less_than_or_equal, greater_than_or_equal

'example7.bnf' is a simple grammar just to check if everything is working well.

Write your own grammar in a text file and save it in your Drive account.

Put the whole address on GRAMMAR_FILE and print to check it.

In [ ]:
#GRAMMAR_FILE = '/content/drive/MyDrive/data/example.bnf' #put the whole address of your own grammar and remove the # in the beginning of this line
GRAMMAR_FILE = 'example7.bnf' #remove this line when you are using your own grammar

#f = open(GRAMMAR_FILE, "r") #remove the # in the beginning of this line when you are using your own grammar
f = open("grammars/" + GRAMMAR_FILE, "r") #remove this line when you are using your own grammar
print(f.read())
f.close() 


<e>   ::= add(<e>,<e>)|sub(<e>,<e>)|mul(<e>,<e>)|pdiv(<e>,<e>)|x[1]|x[2]|x[3]|x[4]|x[5]|x[6]|x[7]|x[8]|x[9]|x[10]|x[11]|x[12]|x[13]|x[14]|<c><c>.<c><c>
<c>  ::= 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9



x[0] refers to `id` column, so it is not being used by 'example7.bnf'.

If you remove the `id` column, you need to consider it when writing your own grammar.

Run the following cell to put your grammar on the class Grammar.

In [ ]:
#BNF_GRAMMAR = Grammar(GRAMMAR_FILE) #remove the # in the beginning of this line when you are using your own grammar
BNF_GRAMMAR = Grammar(path.join("grammars", GRAMMAR_FILE)) #remove this line when you are using your own grammar

The fitness function is the percentage of outputs wrongly predicted.

You can use any other fitness function, but you need to maintain the comma at the end of the returning lines.

Remember that for now, GRAPE is being used only for minimisation problems, so the fitness function needs to consider it.

In [ ]:
def fitness_eval(individual, points):
    """
    Fitness Function
    """

    x = points[0]
    Y = points[1]
    
    if individual.invalid == True:
        return np.NaN,

    # Evaluate the expression
    try:
        pred = eval(individual.phenotype)
    except (FloatingPointError, ZeroDivisionError, OverflowError,
            MemoryError):
        return np.NaN,
    assert np.isrealobj(pred)
    
    try:
        Y_class = [1 if pred[i] > 0 else 0 for i in range(len(Y))]
    except (IndexError, TypeError):
        return np.NaN,

    compare = np.equal(Y,Y_class)
    fitness = 1 - np.mean(compare)
   
    return fitness,

To use properly the fitness function above with GRAPE, the features must be in the lines, and the samples must be in the columns, so we need to transpose the matrix.

Take a look at the print. If you run this cell two times, the matrix will be transposed again and will not work properly.

In [ ]:
X_train = np.transpose(X_train)
X_test = np.transpose(X_test) 

print('Training (X,Y):\t', X_train.shape, y_train.shape)
print('Test (X):\t', X_test.shape)

Training (X,Y):	 (15, 1000) (1000,)
Test (X):	 (15, 430)


Set the Grammatical Evolution parameters.

In [ ]:
## GE parameters
POPULATION_SIZE = 1000
MAX_GENERATIONS = 200
P_CROSSOVER = 0.8
P_MUTATION = 0.01
ELITE_SIZE = round(0.01*POPULATION_SIZE)
TOURNAMENT_SIZE = 3
HALL_OF_FAME_SIZE = 1
MAX_INIT_TREE_DEPTH = 10
MIN_INIT_TREE_DEPTH = 2
MAX_TREE_DEPTH = 17
MAX_WRAPS = 0
CODON_SIZE = 255

Create a toolbox.

In [ ]:
# create a toolbox for DEAP
toolbox = base.Toolbox()

# define a single objective, minimising fitness strategy:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

creator.create('Individual', ge.Individual, fitness=creator.FitnessMin)

toolbox.register("populationCreator", ge.initialisation_PI_Grow, creator.Individual) 

toolbox.register("evaluate", fitness_eval)

# Tournament selection:
toolbox.register("select", ge.selTournament, tournsize=TOURNAMENT_SIZE)

# Single-point crossover:
toolbox.register("mate", ge.crossover_onepoint)

# Flip-int mutation:
toolbox.register("mutate", ge.mutation_int_flip_per_codon)

In [ ]:
# create initial population (generation 0):
population = toolbox.populationCreator(size=POPULATION_SIZE, 
              bnf_grammar=BNF_GRAMMAR, 
              min_init_tree_depth=MIN_INIT_TREE_DEPTH,
              max_init_tree_depth=MAX_INIT_TREE_DEPTH,
              max_tree_depth=MAX_TREE_DEPTH, 
              max_wraps=MAX_WRAPS,
              codon_size=CODON_SIZE
              )

# define the hall-of-fame object:
hof = tools.HallOfFame(HALL_OF_FAME_SIZE)

# prepare the statistics object:
stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("avg", np.nanmean)
stats.register("std", np.nanstd)
stats.register("min", np.nanmin)
stats.register("max", np.nanmax)

Run Grammatical Evolution.

In [ ]:
population, logbook = algorithms.ge_eaSimpleWithElitism(population, toolbox, cxpb=P_CROSSOVER, mutpb=P_MUTATION,
                      ngen=MAX_GENERATIONS, elite_size=ELITE_SIZE,
                      bnf_grammar=BNF_GRAMMAR, codon_size=CODON_SIZE, 
                      max_tree_depth=MAX_TREE_DEPTH, max_wraps=MAX_WRAPS,
                      points_train=[X_train, y_train], 
                      stats=stats, halloffame=hof, verbose=True
                      )

gen	invalid	avg     	std      	min  	max 	best_ind_length	avg_length	max_length	selection_time	generation_time
0  	0      	0.250643	0.0534016	0.226	0.77	4              	45.676    	229       	0             	1.05449        
1  	334    	0.248468	0.0379415	0.226	0.77	4              	45.352    	193       	0.0087564     	2.55683        
2  	108    	0.251581	0.0553547	0.226	0.756	4              	39.707    	153       	0.0079217     	0.826865       
3  	69     	0.252622	0.0595899	0.226	0.775	4              	42.45     	159       	0.00760627    	0.811168       
4  	48     	0.250598	0.0544379	0.226	0.788	4              	41.839    	150       	0.00705099    	0.69063        
5  	41     	0.247875	0.0486676	0.226	0.77 	4              	41.716    	150       	0.00713658    	0.61828        
6  	11     	0.250001	0.0700594	0.225	0.77 	96             	42.221    	140       	0.00732684    	0.63036        
7  	8      	0.245929	0.0662541	0.225	0.77 	96             	43.174    	139       	0.00854897    	0.555969   

Show the best individual as an expression. The final fitness isn't entirely awful, given that we are ignoring a significant amount of the data, so you should consider this a baseline that you want to exceed.

In [ ]:
# Best individual
import textwrap
best = hof.items[0].phenotype
print("Best individual: \n","\n".join(textwrap.wrap(best,80)))
print("\nTraining Fitness: ", hof.items[0].fitness.values[0])
print("Depth: ", hof.items[0].depth)
print("Length of the genome: ", len(hof.items[0].genome))
print(f'Used portion of the genome: {hof.items[0].used_codons/len(hof.items[0].genome):.2f}')

Best individual: 
 add(add(x[12],add(add(x[12],x[5]),add(add(x[12],x[12]),add(add(add(x[12],add(x[1
2],add(x[13],mul(x[11],add(x[12],x[12]))))),add(add(add(add(x[12],x[11]),add(x[1
2],add(x[13],mul(x[11],add(x[12],x[11]))))),x[12]),x[12])),add(x[12],add(x[12],a
dd(add(add(x[1],add(add(add(x[12],x[12]),x[3]),x[1])),add(add(add(x[12],x[12]),x
[12]),add(x[12],x[5]))),add(x[12],x[11])))))))),add(add(x[12],add(add(add(add(x[
12],x[12]),x[12]),add(x[12],x[5])),add(x[12],x[11]))),add(add(x[12],add(x[12],ad
d(mul(x[12],x[12]),x[12]))),x[5])))

Training Fitness:  0.18500000000000005
Depth:  15
Length of the genome:  112
Used portion of the genome: 0.85


Define a function to predict values. We will need this to produce the predictions that will be uploaded to Kaggle.

In [ ]:
def predict(individual, X):
    x = X
    
    if individual.invalid == True:
        return np.NaN,

    # Evaluate the expression
    try:
        pred = eval(individual.phenotype)
    except (FloatingPointError, ZeroDivisionError, OverflowError,
            MemoryError):
        return np.NaN,
    assert np.isrealobj(pred)
    
    _, c = x.shape
    
    try:
        Y_class = [1 if pred[i] > 0 else 0 for i in range(c)]
    except (IndexError, TypeError):
        return np.NaN,
   
    return Y_class

Predict the classes of the test set.

In [ ]:
y_pred = predict(hof.items[0], X_test)
print("Predicted classes of the test set: ", y_pred)

Predicted classes of the test set:  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 

Save it in a .csv file and submit it in the [Kaggle competition](https://www.kaggle.com/c/cs6271-kagglecompetition/submit).

The format is as follows:
1. First column is the original `id` column in the test set;
2. Second column is named `class_label` and contains the predictions (only 0's or 1's).

In [ ]:
df_id = df_test['id']
df_class = pd.DataFrame(data=y_pred, columns = ['class_label'])
df_pred = pd.concat([df_id, df_class], axis=1)

df_pred.to_csv('predictions.csv', sep=',', index=False)

## Genetic Programming

In [ ]:
from deap import gp
from deap import algorithms
#base, creator and tools were already imported

Set GP parameters.

In [ ]:
POPULATION_SIZE = 
P_CROSSOVER =
P_MUTATION = 
MAX_GENERATIONS = 
HALL_OF_FAME_SIZE = 

MIN_TREE_HEIGHT = 
MAX_TREE_HEIGHT = 
LIMIT_TREE_HEIGHT = 
MUT_MIN_TREE_HEIGHT = 
MUT_MAX_TREE_HEIGHT = 

SyntaxError: ignored

Next steps:

1. Define a fitness function;
2. Add functions and terminals to `gp.PrimitiveSet`. Maybe you will need to define some functions before that;
3. Create the toolbox;
4. Run GP;
5. Define a function to predict samples using a GP individual;
6. Predict the test set, save a .csv file and submit it in the [Kaggle competition](https://www.kaggle.com/c/cs6271-kagglecompetition/submit) to see if the results are better than when using GRAPE. 

Hint: 
    
    Maybe you will need to change the format of the data on X_train and X_test.

In [ ]:
# your code here

## Summary



Your summary here.